In [7]:
import pandas as pd
from pathlib import Path

In [8]:
cd

C:\Users\Rand Sobczak Jr


In [11]:
# Load paths
hosp_dir = Path(r"C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds\real_mimic\data\mimiciv\_hosp")
icu_dir = Path(r"C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds\real_mimic\data\mimiciv\_icu")


In [12]:
# Load data
patients = pd.read_csv(hosp_dir / "patients.csv", parse_dates=["dod"])
admissions = pd.read_csv(hosp_dir / "admissions.csv", parse_dates=["admittime", "dischtime"])
icustays = pd.read_csv(icu_dir / "icustays.csv", parse_dates=["intime", "outtime"])


In [13]:
# Merge data
df = admissions.merge(patients, on="subject_id", how="left")
df = df.merge(icustays, on=["subject_id", "hadm_id"], how="left")

In [14]:
# Mortality label: death within 30 days after discharge
df['mortality_30d'] = (
    df['dod'].notna() &
    (df['dod'] - df['dischtime']).dt.days.between(0, 30)
).astype(int)

In [15]:
# Optional: only first admission per patient
df.sort_values(by="admittime", inplace=True)
df = df.drop_duplicates(subset="subject_id", keep="first")

In [16]:
# Save output
output_path = Path(r"C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds\real_mimic\output\1a_labeled_cohort.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)

In [17]:
# Display
print(f"Labeled cohort shape: {df.shape}")
df[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'dod', 'mortality_30d']].head()


Labeled cohort shape: (180733, 28)


,subject_id,hadm_id,admittime,dischtime,dod,mortality_30d
302208,16904137,21081215,2105-10-04 17:26:00,2105-10-12 11:11:00,NaT,0
182344,14178262,24686846,2106-02-06 20:18:00,2106-02-07 09:31:00,NaT,0
87660,12024697,20302177,2109-12-14 22:50:00,2110-01-15 14:53:00,2111-05-24,0
233881,15350437,20383396,2110-01-11 08:02:00,2110-01-12 18:45:00,NaT,0
355528,18106347,24305596,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,0
